**Import Packages**


In [ ]:
import os
import shutil
from shutil import copyfile
import random
import zipfile
import glob
import urllib.request
import tarfile
import re
from distutils.dir_util import copy_tree
import numpy as np
import sys

**Select The Model**

In [ ]:
repo_url = 'https://github.com/Tony607/object_detection_demo'

# Number of training steps.
num_steps = 5000  

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {     
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12},
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12},
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8},
    'ssd_mobilenet_v1': {
        'model_name': 'ssd_mobilenet_v1_coco_2018_01_28',
        'pipeline_file': 'ssd_mobilenet_v1_coco.config',
        'batch_size': 6}
}

selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size
batch_size = MODELS_CONFIG[selected_model]['batch_size']

**Clone the object_detection_demo repository.**

In [ ]:
# Clone the object_detection_demo
%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
Cloning into 'object_detection_demo'...
remote: Enumerating objects: 124, done.
remote: Total 124 (delta 0), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (124/124), 11.16 MiB | 5.19 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/object_detection_demo
Already up to date.


**Install required packages**

In [ ]:
# Install required packages

%cd /content
!git clone --quiet https://github.com/tensorflow/models.git                #install models directory
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk    #protobuf used to generate py file from proto files
!pip install -q Cython contextlib2 pillow lxml matplotlib                  #used in utitlity functions
!pip install -q pycocotools                                                #used in model_main.py
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.                     #generate python file from proto files
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py                    #Creates a DetectionModel from proto files

/content
Selecting previously unselected package python-bs4.
(Reading database ... 145113 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpa

**Download The Dataset**

In [ ]:
# DataSet

# AlexOnly_1280x720: https://www.kaggle.com/spaceengineer1/bumpdataset-alexonly-withoutnight-1280x720
# AlexOnly_Greyscale (Selected Dataset): https://www.kaggle.com/spaceengineer1/alexonly-greyscale/data#

!rm -rf /content/object_detection_demo/data/images
try:
  os.mkdir('/content/object_detection_demo/data/extracted/')
except OSError:
  pass

!wget --no-check-certificate \
    "https://storage.googleapis.com/kaggle-data-sets/486198/906654/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1581959088&Signature=VAtb%2FftwTnG%2FS1HVqt%2Fnz5i2MiIv%2B0gaylnCvyKWIRsl2oaIUmDJU5JbKrKvObdTO6rP6LBamBBNiTOr1YN35DDqPTV4if%2BwmOYCrzmr3UhY96XdkSK2eHF%2FWr1WpCe2Vzv1T%2F6yhyn6wMxsP3iv%2BfDQJSD2avv3bpPEkvQFUWi2VM4WOjDzVaSouphYUr9sXkom1YqvAtn2Kd0qzRPizj4h4y7J1byOnjgL5PuqjnwY9XqMF18LzKA8Rfg8evVCTZVs2vQUlZj4WLprlONIdJAHOVcz52rqnmavxGB34MtbzNcQhV7B11f4kG3g56cf3Tfq4hr6C10iJ3aIMyyCUg%3D%3D&response-content-disposition=attachment%3B+filename%3Dalexonly-greyscale.zip" \
    -O "/tmp/bumps_dataset.zip"

local_zip = '/tmp/bumps_dataset.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/object_detection_demo/data/')
zip_ref.close()

%cd {repo_dir_path}

!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

test_record_fname = '/content/object_detection_demo/data/annotations/test.record'
train_record_fname = '/content/object_detection_demo/data/annotations/train.record'
label_map_pbtxt_fname = '/content/object_detection_demo/data/annotations/label_map.pbtxt'

--2020-02-17 15:34:24--  https://storage.googleapis.com/kaggle-data-sets/486198/906654/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1581959088&Signature=VAtb%2FftwTnG%2FS1HVqt%2Fnz5i2MiIv%2B0gaylnCvyKWIRsl2oaIUmDJU5JbKrKvObdTO6rP6LBamBBNiTOr1YN35DDqPTV4if%2BwmOYCrzmr3UhY96XdkSK2eHF%2FWr1WpCe2Vzv1T%2F6yhyn6wMxsP3iv%2BfDQJSD2avv3bpPEkvQFUWi2VM4WOjDzVaSouphYUr9sXkom1YqvAtn2Kd0qzRPizj4h4y7J1byOnjgL5PuqjnwY9XqMF18LzKA8Rfg8evVCTZVs2vQUlZj4WLprlONIdJAHOVcz52rqnmavxGB34MtbzNcQhV7B11f4kG3g56cf3Tfq4hr6C10iJ3aIMyyCUg%3D%3D&response-content-disposition=attachment%3B+filename%3Dalexonly-greyscale.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243959824 (233M) [application/zip]
Saving to: ‘/tmp/bumps_dataset.zip’

/tmp/bumps_dataset. 100%[===========

**Download Base Model**

In [ ]:
# Download base model

%cd /content/models/research

MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint   #to print the path of fine_tune_checkpoint

/content/models/research/pretrained_model
total 135M
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 .
drwxr-xr-x 66 root   root  4.0K Feb 17 15:35 ..
-rw-r--r--  1 345018 89939   77 Mar 30  2018 checkpoint
-rw-r--r--  1 345018 89939  67M Mar 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 89939  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 89939  15K Mar 30  2018 model.ckpt.index
-rw-r--r--  1 345018 89939 3.4M Mar 30  2018 model.ckpt.meta
-rw-r--r--  1 345018 89939 4.2K Mar 30  2018 pipeline.config
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 saved_model


'/content/models/research/pretrained_model/model.ckpt'

**Configuring a Training Pipeline**

In [ ]:
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

#get number of classes from label map
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)

with open(pipeline_fname) as f:
    s = f.read()

#edit config file with our hyperparameters    
with open(pipeline_fname, 'w') as f:
    
    s = re.sub('fine_tune_checkpoint: ".*?"','fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)  # fine_tune_checkpoint
    s = re.sub('(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)  # train tfrecord file.
    s = re.sub('(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)     # test tfrecord files.
    s = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)            # label_map_path
    s = re.sub('batch_size: [0-9]+','batch_size: {}'.format(batch_size), s)                                 # Set training batch_size.
    s = re.sub('num_steps: [0-9]+','num_steps: {}'.format(num_steps), s)                                    # Set training steps, num_steps
    s = re.sub('num_classes: [0-9]+','num_classes: {}'.format(num_classes), s)                              # Set number of classes num_classes.
    
    f.write(s)

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

 
copy_tree("/content/models/research/training/", "/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/") 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/model.ckpt-115063.data-00000-of-00001',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/events.out.tfevents.1580382498.5360f0ae44c8',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/model.ckpt-149992.data-00000-of-00001',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/model.ckpt-155342.data-00000-of-00001',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/events.out.tfevents.1580372901.5360f0ae44c8',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/events.out.tfevents.1581793620.c723b17efbb3',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/model.ckpt-115063.index',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/model.ckpt-118367.data-00000-of-00001',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/events.out.tfevents.1580560826.712e43bc1923',
 '/content/drive/My Drive/Detection Output/MobNv2GrSc_FT003/events.out.tfevents.1581780142

**Fine Tuning Hyperparameters**

In [ ]:
# Fine Tunning -- 🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶🎶


"""

# To Change -->

!!!!!!!!!!! means it's important to change this hyperparameter

# faster_rcnn_box_coder           Decrease, Doesn't Affect Much
# image_resizer                   Keep as is,  High loss and very Low mAP at beginning but restored in the End
# dropout_keep_probability        Decrease, Doesn't Affect Much
# use_dropout                     If True: Good Loss, Very Bad mAP
# l2_regularizer                  Keep as is, Doesn't Affect Much
# batch_norm decay                Keep as is, Doesn't Affect Much
# feature_extractor min_depth     Increase (+4) !!!!!!!!!!!
# initial_learning_rate           Decrease (original: 0.004)  !!!!!!!!!!!
# decay_steps                     Increase, but Doesn't Affect Much
# decay_factor                    Increase, Affects Much  !!!!!!!!!!!
# momentum_optimizer_value        Increase Affects Much,  !!!!!!!!!!!
# momentum_optimizer_value decay  Increase Affects Much,  !!!!!!!!!!!         
# apply_sigmoid_to_scores         If True: very bad loss, but slightly better mAP
# batch_norm epsilon              Keep as is, Doesn't Affect Much
# train_config epsilon            Keep as is
# box_code_size                   Decrease, but Doesn't Affect Much
# batch_size                      Increase, Affects Much !!!!!!!!!!!
# hard_example_miner iou_threshold         Decreasing: Very low loss, same mAP !!!!!!!!!!!
# hard_example_miner num_hard_examples     Increase, Affects Much   !!!!!!!!!!!
# max_negatives_per_positive               Doesn't Affect Much
# max_detections_per_class                 Doesn't Affect Much


"""

initial_learning_rate = 0.0008    # 0.004 --
decay_factor = 0.97               # 0.95 ++
momentum_optimizer_value =0.93    # 0.9 ++
num_hard_examples = 3000          # 3000 ++
batch_size = 12                   # 6, 12 ++

iou_threshold = 0.95              # 0.99 --   Manual Change (in hard_example_miner)
decay = 0.95                      # 0.9 ++    Manual Change (in momentum_optimizer_value)
#min_depth = 20                    # 16 +4     Manual Change (in feature_extractor)

with open(pipeline_fname) as f:
    s = f.read()    
with open(pipeline_fname, 'w') as f:

    s = re.sub('initial_learning_rate: [0-9].+', 'initial_learning_rate: {}'.format(initial_learning_rate), s)
    s = re.sub('decay_factor: [0-9].+', 'decay_factor: {}'.format(decay_factor), s)
    s = re.sub('momentum_optimizer_value: [0-9].+', 'momentum_optimizer_value: {}'.format(momentum_optimizer_value), s)
    

    f.write(s)

!cat {pipeline_fname}   #read pipeline file

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_

**Train The Model**

In [ ]:
# Training -- 🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂🚂

num_steps = num_steps + 5000        #to continue training from the last checkpoint

!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0217 15:37:27.987405 140570767890304 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0217 15:37:27.990781 140570767890304 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0217 15:37:27.990929 140570767890304 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0217 15:37:27.991056 140570767890304 module_wrapper.py:139] From /content/models/resea

In [ ]:
!ls {model_dir}  

**Exporting a Trained Inference Graph**

In [ ]:
# Exporting a Trained Inference Graph

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
# --------------------------------------------------------------🎥🎥 Video Test 🎥🎥--------------------------------------------------------------

from google.colab import drive
drive.mount('/content/drive/')

output_directory = './fine_tuned_model'
!ls {output_directory}

In [ ]:
# Functions to be used in detection

PATH_TO_CKPT = '/content/models/research/fine_tuned_model/frozen_inference_graph.pb'               # Path to the .pb file
PATH_TO_LABELS = label_map_pbtxt_fname   # Path to the label_map

assert os.path.isfile(PATH_TO_CKPT)
assert os.path.isfile(PATH_TO_LABELS)

%cd /content/models/research/object_detection  # Change base directory

import cv2
import six.moves.urllib as urllib
import sys
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")                                   
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
#%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

detection_graph = tf.Graph()

with detection_graph.as_default():                     
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):         # Function to convert image into numpy array (but not used in detection)
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

def run_inference_for_single_image(image, graph):     # Detection function called in the last cell to perform detection on each frame
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

**Test the Model on the Video**

In [ ]:
from object_detection.utils import visualization_utils as vis_util

sys.stdout.flush()

frame_width = 1280
frame_height = 720

for k in ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18']:

  TestedOn = 'MobNv1GrSc_FT'
  name = 'Agamy_OnFoot_0' + k

  outputName = '/content/' + name + '_' + TestedOn + '.avi'

  cam = cv2.VideoCapture('/content/drive/My Drive/Bump_Videos/' + name + '.mp4')
  out = cv2.VideoWriter(outputName, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
  frames = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))

  i=0

  while(True): 
    ret,image = cam.read()    # read an image (frame) from the video (one by one)

    if ret:                   # If video isn't finished keep processing
      output_dict = run_inference_for_single_image(image, detection_graph)         # Run inference for frames (one by one)

      vis_util.visualize_boxes_and_labels_on_image_array(
      image,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=4)        # Specify the thickness of the box

      out.write(image)         # Add the output image to the video

      sys.stdout.flush()
      print(str(int(i/frames *100)) +'%-',end='\n',file=sys.stdout, flush=True)
      i=i+1
    else: 
        break

  # Release all space and windows once done 
  cam.release() 
  out.release()
  cv2.destroyAllWindows() 

  shutil.move(outputName,'/content/drive/My Drive/Detection Output')   #save the model on Google Drive